# 分类与聚类
## 分类
### 朴素贝叶斯
$P(C|D)=\frac{P(D|C)P(C)}{P(D)}=\frac{P(D|C)P(C)}{\sum_{c \in C}P(D|C=c)P(C=c)}$

垃圾邮件分类场景：

- D：文档
- C：类别（垃圾邮件或者非垃圾邮件）

给定文档d（随机变量D的一个输出）以及类别集合$C=c_1,\cdots,c_N$（随机变量C的输出），可以用贝叶斯规则计算文档d属于每一个类别$c_i$的可能性$P(c_1|d),\cdots,P(c_N|d)$，然后可以将文档d标注为概率最大的那一类。对文档d的贝叶斯分类：

$Class(d)=arg\ max_{c \in C}P(c|d)=arg\ max_{c \in C}\frac{P(d|c)P(c)}{\sum_{c \in C}P(d|c)P(c)}$

估计先验概率P(c):$P(c)=\frac{N_c}{N}$

- $N_c$：训练样本中已经标注为c的样例
- N：训练样本总数

估计P(d|c):

简化（naive）：文档d可以归结为元素（term）的集合。假设元素（term）之间彼此是独立的。$P(d|c)=\prod_{i=1}^n P(w_i|c)$

因此，需要估计在词表V中的每个词在每个类c的概率$P(w_i|c)$

P(c|d)改写为：$P(c|d)=\frac{P(d|c)P(c)}{\sum_{c \in C}P(d|c)P(c)}=\frac{\prod_{i=1}^VP(w_i|c)P(c)}{\sum_{c \in C}\prod_{i=1}^V P(w_i|c)P(c)}$

剩下的事情就是描述如何估计P(w|c)。

#### 1.多重伯努利模型
给定一个类c，为在词表中的每一个term定义一个二值随机变量$w_i$。概率$P(w_i=1|c)$可以解释为“类别c生成wi的概率”。$P(w_i=0|c)$可以解释为“类别c不生成wi的概率”。

用最大似然估计来估计概率：$P(w|c)=\frac{df_{w,c}}{N_c}$

- $df_{w,c}$是在类别c中包含w的文本数量
- $N_c$是训练样本中类别c的文本数量

采用多重伯努利模型，文档的似然估计P(d|c)写为：$P(d|c)=\prod_{w \in V} P(w|c)^{\delta(w,d)}(1-P(w|c))^{1-\delta(w,d)}$

- $\delta(w,D)$为1，当且仅当w在文档d中。

训练数据稀疏->零概率问题->贝叶斯平滑：$P(w|c)=\frac{df_{w,c}+\alpha_w}{N_c+\alpha_w+\beta_w}$

- $\alpha_w,\beta_w$是依赖于w的参数。
- 一种选择是对所有的w设置$\alpha_w=1,\beta_w=0$，得到估计：$P(w|c)=\frac{df_{w,c}+1}{N_c+1}$
- 另一个选择是对所有w设置$\alpha_w=\mu \frac{N_w}{N},\beta_w=\mu(1-\frac{N_w}{N})$，这里$N_w$是训练文本中包含w的所有文本数，$\mu$是可调参数。得到估计：$P(w|c)=\frac{df_{w,c}+\mu \frac{N_w}{N}}{N_c+\mu}$

#### 2.多项式模型
多重伯努利模型假定词的频率是二值的（“词出现”、“词不出现”），多项式模型假定词的出现是零或者多次（“词出现0次”、“词出现1次”...）。

最大似然估计：$P(w|c)=\frac{tf_{w,c}}{|c|}$

- $tf_{w,c}$是训练集中类别c中w出现的次数。|c|表示训练集中类别c中词的总数。

由于词是根据多项式分布来描述，对于给定类别c，文本的最大似然估计：$P(d|c)=P(|d|)(tf_{w_1,d},tf_{w_2,d},\cdots,tf_{w_V,d})!\prod_{w \in V} P(w|c)^{tf_{w,d}} \propto  \prod_{w \in V} P(w|c)^{tf_{w,d}}$

- $tf_{w,d}$是文档d中w的出现次数
- |d|是文档d中的总词数
- P(|d|)是长度为|d|的文本的生成概率
- $(tf_{w_1,d},tf_{w_2,d},\cdots,tf_{w_V,d})$是多项式系数
- P(|d|)及多项式系数是依赖于文档的，对于分类，可以忽略

词似然的贝叶斯平滑估计：$P(w|c)=\frac{tf_{w,c}+\alpha_w}{|c|+\sum_{w \in V}\alpha_w}$

- $\alpha_w$是依赖于w的参数
- 对所有的w设置$\alpha_w=1$，得到估计：：$P(w|c)=\frac{tf_{w,c}+1}{|c|+|V|}$
- 设置$\alpha_w=\mu\frac{cf_w}{|C|}$，这里$cf_w$是在训练集中w出现的次数，|C|是训练集中词的总数，$\mu$是可调参数，得到估计：：$P(w|c)=\frac{tf_{w,c}+\mu\frac{cf_w}{|C|}}{|c|+\mu}$，即Dirichlet平滑语言模型估计

### SVM
SVM的目标是发现一个分离正例和反例的超平面。

#### 1.线性可分数据
最大化分离定义：假设$x^-$是训练集中与超平面最近的反例，$x^+$是训练集中与超平面最近的正例，定义边缘（margin）为$x^-$到超平面加上$x^+$到超平面的距离。边缘可以计算如下：

$Margion(w)=\frac{|w \cdot x^-|+(w \cdot x^+)}{\lVert w \rVert}$

为了简化问题，通常假设$w \cdot x^- =-1,w \cdot x^+=1$，这个假设不能改变对问题的解法，使边缘等于$\frac{2}{\lVert w \rVert}$。另一个等价的公式：

$min:\ \frac{1}{2}\lVert w \rVert^2$

subject to:

$w \cdot x_i \ge 1 \forall i\ s.t.\ Class(i)=+$

$w \cdot x_i \le -1 \forall i\ s.t.\ Class(i)=-$

#### 2.线性不可分
惩罚因子（关键损失函数）：

$
L(x)=\left\{\begin{matrix}
 max(1-w \cdot x,0)\ if\ Class(i)=+ \\ 
 max(1+w \cdot x,0)\ if\ Class(i)=-
\end{matrix}\right.
$

这个损失函数合并进SVM最优化如下：

$min:\ \frac{1}{2}\lVert w \rVert^2+C\sum_{i=1}^N \xi_i$

subject to:

$w \cdot x_i \ge 1-\xi_i\ \forall i\ s.t.\ Class(i)=+$

$w \cdot x_i \le -1+\xi_i\ \forall i\ s.t.\ Class(i)=-$

$\xi_i \ge 0\ \forall i $

这里$\xi_i$称为允许目标值被违反的松弛变量。

#### 3.核技巧
在线性不可分的情况下，将它们变换或映射到高维空间，得到线性可分的点的集合。

#### 4.非二分类
- One vs All，OVA：假设K大于2，训练K个分类器，每次将第k类看作正例，其他所有类看出反例。x被划分到的类别是分类器的值$w \cdot x$中最大的那一类。
- One vs One，OVO：对所有的分类K训练全组合个分类器（K(K-1)/2），每一次x被划分到c类，对c的投票被记录，最后，x被划分到具有最多投票的类别中。

### 评价
详见第8章。

### 分类器和特征选择
1. 生成模型
    1. 贝叶斯分类器
    1. 首先根据P(c)生成一个类，然后根据P(d|c)生成文档。生成模型模仿人类实际生成（撰写）文档的方式。
1. 判别模型
    1. SVM分类器
    1. 不会去模拟文本和类别的生成过程，而是直接模拟对给定的输入文本赋予类别。
1. 非参数分类器
    1. knn分类器
    1. 去除所有分布假设让数据“自己说话”。
1. 特征选择
    1. 目的是在最初的特征空间中，找到能够代表原特征空间的主要特征的特征子集，这既能提高效率同时又不会过多地影响其有效性。
    1. 信息增益（information gain）是文本分类中应用最广泛的特征选择方法。
    1. 特征f的信息增益度量的实现f之后P(C)的熵的变化：$IG(w)=H(C)-H(C|w)=-\sum_{c \in C}P(c)log P(c)+\sum_{w \in\{0,1\}}P(w)\sum_{c \in C}P(c|w)logP(c|w)$
        1. H(C)是P(C)的熵
        1. H(C|w)称为条件熵

### 垃圾、情感及在线广告
1. 垃圾
    1. 垃圾邮件
    1. 垃圾广告
    1. 垃圾网页
        1. 链接垃圾：垃圾信息制造者用各种办法人为地提高他们网页的链接分值。
        1. 词垃圾：试图改变文本的表示，以便能被某些查询或者关键词检索。
            1. 倾销：用许多不相关的词（常常是整个词典）填充文本，导致文本被任何查询检索，因为几乎包含了查询词的每一种组合，这起到了加强召回率的作用。
            1. 短语拼接：从多种来源合并词和句子。
            1. 编织：将垃圾词加入到有效的文本中，如新闻故事。
1. 情感：词汇线索确定感情。
    1. 正面情感指示词：“great”、“nice”、“amazing”
    1. 负面情感指示词：“awful”、“terrible”、“bad”
1. 分类广告
    1. 将查询或网页正文的文本项划分为语义层次，这个层次由手工构建，包括6000多个节点，每一个节点代表一个独立的语义类，层次越深，类别约明确。
    1. 对广告清单中的每一个广告进行预分类。

## 聚类
### 层次聚类和K均值聚类
层次聚类分为两类：

1. 分裂聚类：自顶向下的方法，从一个包含所有实例的簇开始。每一次迭代时，它从现有的簇中选择一个簇分成两个簇。不断进行这个过程，直到总共产生K个簇。
1. 聚合聚类：自底向上的方法。每次将两个现有的簇合并成一个新的簇。当剩余K个簇时，算法停止。

K均值聚类：保持K个簇不变，每次迭代中，每个实例要么保留在原有簇中，要么被分配到一个其他的簇中。这个过程不断重复，直到满足停止要求。

### K近邻聚类
层次聚类和K均值聚类的特点是：簇之间不重叠。K近邻聚类，围绕每个实例形成一个簇。对于输入实例x，根据某个距离衡量标准，距离x最近的K个点和x本身构成一个簇。实际上每一个输入实例都对应一个簇，因此会产生N个簇。